In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
pd.options.display.max_rows

60

In [60]:
kp = pd.read_csv("/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/KenPom Team Metrics.csv")
#kp.head(10)
kp_conf_rat = pd.read_csv("/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/KP_Conf_Ratings.csv")
kp_conf_rat = kp_conf_rat[["Year_Conf", "Conf_Abb", "Conf_Rk", "Conf_Full", "Conf_Rating"]]
kp_conf_rat.head(10)

,Year_Conf,Conf_Abb,Conf_Rk,Conf_Full,Conf_Rating
0,2023,Big 12,1,Big 12 Conference,17.99
1,2023,Big Ten,2,Big Ten Conference,14.88
2,2023,Big East,3,Big East Conference,13.95
3,2023,SEC,4,Southeastern Conference,12.57
4,2023,Pac-12,5,Pac 12 Conference,11.58
5,2023,MWC,6,Mountain West Conference,9.26
6,2023,ACC,7,Atlantic Coast Conference,8.58
7,2023,AAC,8,American Athletic Conference,6.72
8,2023,WCC,9,West Coast Conference,6.27
9,2023,CUSA,10,Conference USA,3.60


In [65]:
d = pd.read_csv("/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/final_data_by_player_type.csv")
d = pd.merge(d, kp, left_on=['Team', 'year'], right_on=['TeamName', 'Season'], how='left')
d = pd.merge(d, kp, left_on=['Team_n1', 'year'], right_on=['TeamName', 'Season'], how='left')
df = pd.merge(d, kp, left_on=['Team_n1', 'year_n1'], right_on=['TeamName', 'Season'], how='left')
df = pd.merge(df, kp_conf_rat, left_on=['conf_abbr', 'year'], right_on=['Conf_Abb', 'Year_Conf'], how='left')
df = pd.merge(df, kp_conf_rat, left_on=['conf_abbr_n1', 'year'], right_on=['Conf_Abb', 'Year_Conf'], how='left')
df = pd.merge(df, kp_conf_rat, left_on=['conf_abbr_n1', 'year_n1'], right_on=['Conf_Abb', 'Year_Conf'], how='left')
df = df.replace(to_replace="NO IDEA", value=0)
df = df.replace(to_replace="---", value=0)
df = df.fillna(0)
pd.options.display.max_columns
#pd.options.display.max_rows
df



/var/folders/cq/16hkxgp926d466c0g9jzx2900000gq/T/ipykernel_66368/1518560658.py:1: DtypeWarning: Columns (13,16,38,144,147,169) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv("/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/final_data_by_player_type.csv")


,Unnamed: 0,player_type,Team,Team_n1,Unnamed..0_x,Player.Name,season,conf_abbr,games,games_started,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg2_per_g,fg2a_per_g,fg2_pct,fg3_per_g,fg3a_per_g,fg3_pct,ft_per_g,fta_per_g,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pf_per_g,pts_per_g,sos,mp,fg,fga,fg2,fg2a,fg2_pct.1,fg3,fg3a,ft,fta,orb,drb,trb,ast,stl,blk,tov,pf,pts,fg_per_min,fga_per_min,fg2_per_min,fg2a_per_min,fg3_per_min,fg3a_per_min,ft_per_min,fta_per_min,trb_per_min,ast_per_min,stl_per_min,blk_per_min,tov_per_min,pf_per_min,pts_per_min,fg_per_poss,fga_per_poss,fg2_per_poss,fg2a_per_poss,fg3_per_poss,fg3a_per_poss,ft_per_poss,fta_per_poss,trb_per_poss,ast_per_poss,stl_per_poss,blk_per_poss,tov_per_poss,pf_per_poss,pts_per_poss,off_rtg,def_rtg,per,ts_pct,efg_pct,fg3a_per_fga_pct,fta_per_fga_pct,pprod,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,ows,dws,ws,ws_per_40,obpm,dbpm,bpm,year,Ht,Wt,Yr,G,S,X.Min,ORtg,X.Poss,X.Shots,eFG.,TS.,OR.,DR.,ARate,TORate,Blk.,Stl.,FC.40,FD.40,FTRate,FTM.A,Pct,X2PM.A,Pct.1,X3PM.A,Pct.2,X.Pct,X.2PM.A,X.Pct.1,Unnamed..0_x_n1,Player.Name_n1,season_n1,conf_abbr_n1,games_n1,games_started_n1,mp_per_g_n1,fg_per_g_n1,fga_per_g_n1,fg_pct_n1,fg2_per_g_n1,fg2a_per_g_n1,fg2_pct_n1,fg3_per_g_n1,fg3a_per_g_n1,fg3_pct_n1,ft_per_g_n1,fta_per_g_n1,ft_pct_n1,orb_per_g_n1,drb_per_g_n1,trb_per_g_n1,ast_per_g_n1,stl_per_g_n1,blk_per_g_n1,tov_per_g_n1,pf_per_g_n1,pts_per_g_n1,sos_n1,mp_n1,fg_n1,fga_n1,fg2_n1,fg2a_n1,fg2_pct.1_n1,fg3_n1,fg3a_n1,ft_n1,fta_n1,orb_n1,drb_n1,trb_n1,ast_n1,stl_n1,blk_n1,tov_n1,pf_n1,pts_n1,fg_per_min_n1,fga_per_min_n1,fg2_per_min_n1,fg2a_per_min_n1,fg3_per_min_n1,fg3a_per_min_n1,ft_per_min_n1,fta_per_min_n1,trb_per_min_n1,ast_per_min_n1,stl_per_min_n1,blk_per_min_n1,tov_per_min_n1,pf_per_min_n1,pts_per_min_n1,fg_per_poss_n1,fga_per_poss_n1,fg2_per_poss_n1,fg2a_per_poss_n1,fg3_per_poss_n1,fg3a_per_poss_n1,ft_per_poss_n1,fta_per_poss_n1,trb_per_poss_n1,ast_per_poss_n1,stl_per_poss_n1,blk_per_poss_n1,tov_per_poss_n1,pf_per_poss_n1,pts_per_poss_n1,off_rtg_n1,def_rtg_n1,per_n1,ts_pct_n1,efg_pct_n1,fg3a_per_fga_pct_n1,fta_per_fga_pct_n1,pprod_n1,orb_pct_n1,drb_pct_n1,trb_pct_n1,ast_pct_n1,stl_pct_n1,blk_pct_n1,tov_pct_n1,usg_pct_n1,ows_n1,dws_n1,ws_n1,ws_per_40_n1,obpm_n1,dbpm_n1,bpm_n1,year_n1,Ht_n1,Wt_n1,Yr_n1,G_n1,S_n1,X.Min_n1,ORtg_n1,X.Poss_n1,X.Shots_n1,eFG._n1,TS._n1,OR._n1,DR._n1,ARate_n1,TORate_n1,Blk._n1,Stl._n1,FC.40_n1,FD.40_n1,FTRate_n1,FTM.A_n1,Pct_n1,X2PM.A_n1,Pct.1_n1,X3PM.A_n1,Pct.2_n1,X.Pct_n1,X.2PM.A_n1,X.Pct.1_n1,Season_x,TeamName_x,TeamName 2_x,Tempo_x,RankTempo_x,AdjTempo_x,RankAdjTempo_x,OE_x,RankOE_x,AdjOE_x,RankAdjOE_x,DE_x,RankDE_x,AdjDE_x,RankAdjDE_x,AdjEM_x,RankAdjEM_x,seed_x,Season_y,TeamName_y,TeamName 2_y,Tempo_y,RankTempo_y,AdjTempo_y,RankAdjTempo_y,OE_y,RankOE_y,AdjOE_y,RankAdjOE_y,DE_y,RankDE_y,AdjDE_y,RankAdjDE_y,AdjEM_y,RankAdjEM_y,seed_y,Season,TeamName,TeamName 2,Tempo,RankTempo,AdjTempo,RankAdjTempo,OE,RankOE,AdjOE,RankAdjOE,DE,RankDE,AdjDE,RankAdjDE,AdjEM,RankAdjEM,seed,Year_Conf_x,Conf_Abb_x,Conf_Rk_x,Conf_Full_x,Conf_Rating_x,Year_Conf_y,Conf_Abb_y,Conf_Rk_y,Conf_Full_y,Conf_Rating_y,Year_Conf,Conf_Abb,Conf_Rk,Conf_Full,Conf_Rating
0,1,1,JACKSONVILLE,JACKSONVILLE,16199.0,aamahne-santos-1,2018-19,A-Sun,32,31,28.6,2.3,6.3,0.361,0.9,2.3,0.4,1.3,4.0,0.339,0.9,0.9,0.966,0.3,1.5,1.8,2.4,0.6,0.1,1.5,1.3,6.8,-4.74,915,73.0,202.0,30.0,75.0,0.4,43.0,127.0,28.0,29.0,10.0,47.0,57,78.0,20.0,2.0,49.0,43.0,217.0,3.2,8.8,1.3,3.3,1.9,5.6,1.2,1.3,2.5,3.4,0.9,0.1,2.1,1.9,9.5,4.4,12.0,1.8,4.5,2.6,7.6,1.7,1.7,3.4,4.7,1.2,0.1,2.9,2.6,12.9,98.5,109.4,7.9,0.503,0.468,0.629,0.144,227.0,1.2,5.8,3.5,13.9,1.2,0.2,18.5,13.8,0.5,0.6,1.1,0.048,-2.9,-1.7,-4.7,2019.0,70,165.0,Jr,30,29.0,73.4,94.5,13.8,14.6,44.3,48.1,1.2,5.7,13.6,20.8,0.2,1.2,1.9,1.9,15.0,28-29,0.966,27-72,0.375,39-121,0.322,0.966,27-72,0.375,16200.0,aamahne-santos-1,2019-20,A-Sun,30,29,29.6,2.9,7.7,0.372,1.5,3.6,0.413,1.4,4.1,0.336,0.8,1.1,0.719,0.2,2.1,2.3,2.6,0.8,0.0,1.7,1.5,

In [66]:
team_df = df[["Team", "Team_n1"]]
team_df_n = team_df[["Team"]]
team_df_n = team_df_n.sort_values(by=["Team"])

team_df_n1 = team_df[["Team_n1"]]
team_df_n1 = team_df_n1.sort_values(by=["Team_n1"])

#print(len(team_df.Team.unique()))
#print(list(team_df_n1.Team_n1.unique()))

In [67]:
# Converts to numeric
df[['games', 'games_started', 'mp_per_g', 'fg_per_g', 'fga_per_g', 'fg_pct', 'fg2_per_g', 'fg2a_per_g', 'fg2_pct', 'fg3_per_g', 'fg3a_per_g', 'fg3_pct', 'ft_per_g', 'fta_per_g', 'ft_pct', 'orb_per_g', 'drb_per_g', 'trb_per_g', 'ast_per_g', 'stl_per_g', 'blk_per_g', 'tov_per_g', 'pf_per_g', 'pts_per_g', 'sos', 'mp', 'fg', 'fga', 'fg2', 'fg2a', 'fg2_pct.1', 'fg3', 'fg3a', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'fg_per_min', 'fga_per_min', 'fg2_per_min', 'fg2a_per_min', 'fg3_per_min', 'fg3a_per_min', 'ft_per_min', 'fta_per_min', 'trb_per_min', 'ast_per_min', 'stl_per_min', 'blk_per_min', 'tov_per_min', 'pf_per_min', 'pts_per_min', 'fg_per_poss', 'fga_per_poss', 'fg2_per_poss', 'fg2a_per_poss', 'fg3_per_poss', 'fg3a_per_poss', 'ft_per_poss', 'fta_per_poss', 'trb_per_poss', 'ast_per_poss', 'stl_per_poss', 'blk_per_poss', 'tov_per_poss', 'pf_per_poss', 'pts_per_poss', 'off_rtg', 'def_rtg', 'per', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'pprod', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'ows', 'dws', 'ws', 'ws_per_40', 'obpm', 'dbpm', 'bpm', 'year', 'Ht', 'Wt', 'G', 'S', 'X.Min', 'ORtg', 'X.Poss', 'X.Shots', 'games_n1', 'games_started_n1', 'mp_per_g_n1', 'fg_per_g_n1', 'fga_per_g_n1', 'fg_pct_n1', 'fg2_per_g_n1', 'fg2a_per_g_n1', 'fg2_pct_n1', 'fg3_per_g_n1', 'fg3a_per_g_n1', 'fg3_pct_n1', 'ft_per_g_n1', 'fta_per_g_n1', 'ft_pct_n1', 'orb_per_g_n1', 'drb_per_g_n1', 'trb_per_g_n1', 'ast_per_g_n1', 'stl_per_g_n1', 'blk_per_g_n1', 'tov_per_g_n1', 'pf_per_g_n1', 'pts_per_g_n1', 'sos_n1', 'mp_n1', 'fg_n1', 'fga_n1', 'fg2_n1', 'fg2a_n1', 'fg2_pct.1_n1', 'fg3_n1', 'fg3a_n1', 'ft_n1', 'fta_n1', 'orb_n1', 'drb_n1', 'trb_n1', 'ast_n1', 'stl_n1', 'blk_n1', 'tov_n1', 'pf_n1', 'pts_n1', 'fg_per_min_n1', 'fga_per_min_n1', 'fg2_per_min_n1', 'fg2a_per_min_n1', 'fg3_per_min_n1', 'fg3a_per_min_n1', 'ft_per_min_n1', 'fta_per_min_n1', 'trb_per_min_n1', 'ast_per_min_n1', 'stl_per_min_n1', 'blk_per_min_n1', 'tov_per_min_n1', 'pf_per_min_n1', 'pts_per_min_n1', 'fg_per_poss_n1', 'fga_per_poss_n1', 'fg2_per_poss_n1', 'fg2a_per_poss_n1', 'fg3_per_poss_n1', 'fg3a_per_poss_n1', 'ft_per_poss_n1', 'fta_per_poss_n1', 'trb_per_poss_n1', 'ast_per_poss_n1', 'stl_per_poss_n1', 'blk_per_poss_n1', 'tov_per_poss_n1', 'pf_per_poss_n1', 'pts_per_poss_n1', 'off_rtg_n1', 'def_rtg_n1', 'per_n1', 'ts_pct_n1', 'efg_pct_n1', 'fg3a_per_fga_pct_n1', 'fta_per_fga_pct_n1', 'pprod_n1', 'orb_pct_n1', 'drb_pct_n1', 'trb_pct_n1', 'ast_pct_n1', 'stl_pct_n1', 'blk_pct_n1', 'tov_pct_n1', 'usg_pct_n1', 'ows_n1', 'dws_n1', 'ws_n1', 'ws_per_40_n1', 'obpm_n1', 'dbpm_n1', 'bpm_n1', 'year_n1', 'Ht_n1', 'Wt_n1', 'G_n1', 'S_n1', 'X.Min_n1', 'ORtg_n1', 'X.Poss_n1', 'X.Shots_n1']] = df[['games', 'games_started', 'mp_per_g', 'fg_per_g', 'fga_per_g', 'fg_pct', 'fg2_per_g', 'fg2a_per_g', 'fg2_pct', 'fg3_per_g', 'fg3a_per_g', 'fg3_pct', 'ft_per_g', 'fta_per_g', 'ft_pct', 'orb_per_g', 'drb_per_g', 'trb_per_g', 'ast_per_g', 'stl_per_g', 'blk_per_g', 'tov_per_g', 'pf_per_g', 'pts_per_g', 'sos', 'mp', 'fg', 'fga', 'fg2', 'fg2a', 'fg2_pct.1', 'fg3', 'fg3a', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'fg_per_min', 'fga_per_min', 'fg2_per_min', 'fg2a_per_min', 'fg3_per_min', 'fg3a_per_min', 'ft_per_min', 'fta_per_min', 'trb_per_min', 'ast_per_min', 'stl_per_min', 'blk_per_min', 'tov_per_min', 'pf_per_min', 'pts_per_min', 'fg_per_poss', 'fga_per_poss', 'fg2_per_poss', 'fg2a_per_poss', 'fg3_per_poss', 'fg3a_per_poss', 'ft_per_poss', 'fta_per_poss', 'trb_per_poss', 'ast_per_poss', 'stl_per_poss', 'blk_per_poss', 'tov_per_poss', 'pf_per_poss', 'pts_per_poss', 'off_rtg', 'def_rtg', 'per', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'pprod', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'ows', 'dws', 'ws', 'ws_per_40', 'obpm', 'dbpm', 'bpm', 'year', 'Ht', 'Wt', 'G', 'S', 'X.Min', 'ORtg', 'X.Poss', 'X.Shots', 'games_n1', 'games_started_n1', 'mp_per_g_n1', 'fg_per_g_n1', 'fga_per_g_n1', 'fg_pct_n1', 'fg2_per_g_n1', 'fg2a_per_g_n1', 'fg2_pct_n1', 'fg3_per_g_n1', 'fg3a_per_g_n1', 'fg3_pct_n1', 'ft_per_g_n1', 'fta_per_g_n1', 'ft_pct_n1', 'orb_per_g_n1', 'drb_per_g_n1', 'trb_per_g_n1', 'ast_per_g_n1', 'stl_per_g_n1', 'blk_per_g_n1', 'tov_per_g_n1', 'pf_per_g_n1', 'pts_per_g_n1', 'sos_n1', 'mp_n1', 'fg_n1', 'fga_n1', 'fg2_n1', 'fg2a_n1', 'fg2_pct.1_n1', 'fg3_n1', 'fg3a_n1', 'ft_n1', 'fta_n1', 'orb_n1', 'drb_n1', 'trb_n1', 'ast_n1', 'stl_n1', 'blk_n1', 'tov_n1', 'pf_n1', 'pts_n1', 'fg_per_min_n1', 'fga_per_min_n1', 'fg2_per_min_n1', 'fg2a_per_min_n1', 'fg3_per_min_n1', 'fg3a_per_min_n1', 'ft_per_min_n1', 'fta_per_min_n1', 'trb_per_min_n1', 'ast_per_min_n1', 'stl_per_min_n1', 'blk_per_min_n1', 'tov_per_min_n1', 'pf_per_min_n1', 'pts_per_min_n1', 'fg_per_poss_n1', 'fga_per_poss_n1', 'fg2_per_poss_n1', 'fg2a_per_poss_n1', 'fg3_per_poss_n1', 'fg3a_per_poss_n1', 'ft_per_poss_n1', 'fta_per_poss_n1', 'trb_per_poss_n1', 'ast_per_poss_n1', 'stl_per_poss_n1', 'blk_per_poss_n1', 'tov_per_poss_n1', 'pf_per_poss_n1', 'pts_per_poss_n1', 'off_rtg_n1', 'def_rtg_n1', 'per_n1', 'ts_pct_n1', 'efg_pct_n1', 'fg3a_per_fga_pct_n1', 'fta_per_fga_pct_n1', 'pprod_n1', 'orb_pct_n1', 'drb_pct_n1', 'trb_pct_n1', 'ast_pct_n1', 'stl_pct_n1', 'blk_pct_n1', 'tov_pct_n1', 'usg_pct_n1', 'ows_n1', 'dws_n1', 'ws_n1', 'ws_per_40_n1', 'obpm_n1', 'dbpm_n1', 'bpm_n1', 'year_n1', 'Ht_n1', 'Wt_n1', 'G_n1', 'S_n1', 'X.Min_n1', 'ORtg_n1', 'X.Poss_n1', 'X.Shots_n1']].apply(pd.to_numeric, errors="ignore")


In [68]:
conditions = [(df['X.Poss_n1']>=28.0), (df['X.Poss_n1']>=24.0), (df['X.Poss_n1']>=20.0), (df['X.Poss_n1']>=16), 
              (df['X.Poss_n1']>=12.0), (df['X.Poss_n1']<=12) & (df['X.Min_n1']>=10.0), (df['X.Min_n1']<10)]
#values = ['Go-To Guy', 'Major Contributor', 'Significant Contributor', 'Role Players', 'Limited Role', 'Nearly Invisible', 'Bench Warmer']
values = [0, 1, 2, 3, 4, 5, 6]
df['Contribution_n1'] = np.select(conditions, values)

/var/folders/cq/16hkxgp926d466c0g9jzx2900000gq/T/ipykernel_66368/1823136958.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Contribution_n1'] = np.select(conditions, values)


In [69]:
df_redshirt = df.loc[(df['games']<=10) & (df['games_started']/df['games']>=0.7)]
#& (df.games_started/df.games>=0.7)]
df_redshirt

,Unnamed: 0,player_type,Team,Team_n1,Unnamed..0_x,Player.Name,season,conf_abbr,games,games_started,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg2_per_g,fg2a_per_g,fg2_pct,fg3_per_g,fg3a_per_g,fg3_pct,ft_per_g,fta_per_g,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pf_per_g,pts_per_g,sos,mp,fg,fga,fg2,fg2a,fg2_pct.1,fg3,fg3a,ft,fta,orb,drb,trb,ast,stl,blk,tov,pf,pts,fg_per_min,fga_per_min,fg2_per_min,fg2a_per_min,fg3_per_min,fg3a_per_min,ft_per_min,fta_per_min,trb_per_min,ast_per_min,stl_per_min,blk_per_min,tov_per_min,pf_per_min,pts_per_min,fg_per_poss,fga_per_poss,fg2_per_poss,fg2a_per_poss,fg3_per_poss,fg3a_per_poss,ft_per_poss,fta_per_poss,trb_per_poss,ast_per_poss,stl_per_poss,blk_per_poss,tov_per_poss,pf_per_poss,pts_per_poss,off_rtg,def_rtg,per,ts_pct,efg_pct,fg3a_per_fga_pct,fta_per_fga_pct,pprod,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,ows,dws,ws,ws_per_40,obpm,dbpm,bpm,year,Ht,Wt,Yr,G,S,X.Min,ORtg,X.Poss,X.Shots,eFG.,TS.,OR.,DR.,ARate,TORate,Blk.,Stl.,FC.40,FD.40,FTRate,FTM.A,Pct,X2PM.A,Pct.1,X3PM.A,Pct.2,X.Pct,X.2PM.A,X.Pct.1,Unnamed..0_x_n1,Player.Name_n1,season_n1,conf_abbr_n1,games_n1,games_started_n1,mp_per_g_n1,fg_per_g_n1,fga_per_g_n1,fg_pct_n1,fg2_per_g_n1,fg2a_per_g_n1,fg2_pct_n1,fg3_per_g_n1,fg3a_per_g_n1,fg3_pct_n1,ft_per_g_n1,fta_per_g_n1,ft_pct_n1,orb_per_g_n1,drb_per_g_n1,trb_per_g_n1,ast_per_g_n1,stl_per_g_n1,blk_per_g_n1,tov_per_g_n1,pf_per_g_n1,pts_per_g_n1,sos_n1,mp_n1,fg_n1,fga_n1,fg2_n1,fg2a_n1,fg2_pct.1_n1,fg3_n1,fg3a_n1,ft_n1,fta_n1,orb_n1,drb_n1,trb_n1,ast_n1,stl_n1,blk_n1,tov_n1,pf_n1,pts_n1,fg_per_min_n1,fga_per_min_n1,fg2_per_min_n1,fg2a_per_min_n1,fg3_per_min_n1,fg3a_per_min_n1,ft_per_min_n1,fta_per_min_n1,trb_per_min_n1,ast_per_min_n1,stl_per_min_n1,blk_per_min_n1,tov_per_min_n1,pf_per_min_n1,pts_per_min_n1,fg_per_poss_n1,fga_per_poss_n1,fg2_per_poss_n1,fg2a_per_poss_n1,fg3_per_poss_n1,fg3a_per_poss_n1,ft_per_poss_n1,fta_per_poss_n1,trb_per_poss_n1,ast_per_poss_n1,stl_per_poss_n1,blk_per_poss_n1,tov_per_poss_n1,pf_per_poss_n1,pts_per_poss_n1,off_rtg_n1,def_rtg_n1,per_n1,ts_pct_n1,efg_pct_n1,fg3a_per_fga_pct_n1,fta_per_fga_pct_n1,pprod_n1,orb_pct_n1,drb_pct_n1,trb_pct_n1,ast_pct_n1,stl_pct_n1,blk_pct_n1,tov_pct_n1,usg_pct_n1,ows_n1,dws_n1,ws_n1,ws_per_40_n1,obpm_n1,dbpm_n1,bpm_n1,year_n1,Ht_n1,Wt_n1,Yr_n1,G_n1,S_n1,X.Min_n1,ORtg_n1,X.Poss_n1,X.Shots_n1,eFG._n1,TS._n1,OR._n1,DR._n1,ARate_n1,TORate_n1,Blk._n1,Stl._n1,FC.40_n1,FD.40_n1,FTRate_n1,FTM.A_n1,Pct_n1,X2PM.A_n1,Pct.1_n1,X3PM.A_n1,Pct.2_n1,X.Pct_n1,X.2PM.A_n1,X.Pct.1_n1,Season_x,TeamName_x,TeamName 2_x,Tempo_x,RankTempo_x,AdjTempo_x,RankAdjTempo_x,OE_x,RankOE_x,AdjOE_x,RankAdjOE_x,DE_x,RankDE_x,AdjDE_x,RankAdjDE_x,AdjEM_x,RankAdjEM_x,seed_x,Season_y,TeamName_y,TeamName 2_y,Tempo_y,RankTempo_y,AdjTempo_y,RankAdjTempo_y,OE_y,RankOE_y,AdjOE_y,RankAdjOE_y,DE_y,RankDE_y,AdjDE_y,RankAdjDE_y,AdjEM_y,RankAdjEM_y,seed_y,Season,TeamName,TeamName 2,Tempo,RankTempo,AdjTempo,RankAdjTempo,OE,RankOE,AdjOE,RankAdjOE,DE,RankDE,AdjDE,RankAdjDE,AdjEM,RankAdjEM,seed,Year_Conf_x,Conf_Abb_x,Conf_Rk_x,Conf_Full_x,Conf_Rating_x,Year_Conf_y,Conf_Abb_y,Conf_Rk_y,Conf_Full_y,Conf_Rating_y,Year_Conf,Conf_Abb,Conf_Rk,Conf_Full,Conf_Rating,Contribution_n1
14,15,0,SOUTHERN ILLINOIS,GONZAGA,0.0,aaron-cook-2,2019-20,MVC,6,6,31.2,6.2,11.2,0.552,5.7,9.3,0.607,0.5,1.8,0.273,2.2,3.7,0.591,0.3,2.7,3.0,3.3,3.2,0.2,2.3,2.3,15.0,0.37,187,37.0,67.0,34.0,56.0,0.607,3.0,11.0,13.0,22.0,2.0,16.0,18,20.0,19.0,1.0,14.0,14.0,90.0,7.9,14.3,7.3,12.0,0.6,2.4,2.8,4.7,3.9,4.3,4.1,0.2,3.0,3.0,19.3,12.3,22.3,11.3,18.6,1.0,3.7,4.3,7.3,6.0,6.6,6.3,0.3,4.7,4.7,29.9,106.1,89.6,26.0,0.581,0.575,0.164,0.328,85.0,1.4,10.3,5.9,31.3,6.3,0.6,15.3,27.6,0.4,0.4,0.9,0.186,3.1,6.2,9.3,2020.0,74,185.0,Sr,5,5.0,13.2,105.3,26.3,27.2,59.8,60.2,0.8,7.5,27.2,17.4,0.7,6.8,2.7,4.6,39.3,13-22,0.591,29-48,0.604,3-8,0.375,0.591,29-48,0.604,0.0,aaron-cook-2,2020-21,WCC,30,1,13.4,1.5,3.0,0.500,1.0,1.7,0.620,0.5,1.3,0.350,0.8,1.2,0.657,0.2,1.4,1.6,1.7,0.9,0.1,1.1,1.3,4.2,5

In [70]:
#df = df.drop(df[(df['games']<=12) & (df['games_started']/df['games']>=0.6)].index)
df['Year_dif'] = df['year_n1']-df['year']
df['Team_AdjEM_dif'] = df['AdjEM_y']-df['AdjEM_x']
df['KP_Rk_Dif'] = df['RankAdjEM_y']-df['RankAdjEM_x']
df['Calc A'] = df['X.Min']*df['ORtg']
# Trying to flag red 
df_test1 = df[['player_type', 'X.Poss', 'X.Poss_n1', 'X.Min',
                'X.Min_n1', 'ORtg', 'Contribution_n1', 'Year_dif', 'stl_per_poss', 
                'blk_per_poss', 'DR.', 'KP_Rk_Dif', 'Calc A']]
#'Team_AdjEM_dif'
X = df_test1[['player_type', 'X.Poss', 'X.Min', 'X.Min_n1', 'ORtg', 
                      'Year_dif', 'stl_per_poss', 'blk_per_poss', 
                      'DR.', 'KP_Rk_Dif', 'Calc A']]
y = df_test1[['Contribution_n1']]

/var/folders/cq/16hkxgp926d466c0g9jzx2900000gq/T/ipykernel_66368/3416284705.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Year_dif'] = df['year_n1']-df['year']
/var/folders/cq/16hkxgp926d466c0g9jzx2900000gq/T/ipykernel_66368/3416284705.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Team_AdjEM_dif'] = df['AdjEM_y']-df['AdjEM_x']
/var/folders/cq/16hkxgp926d466c0g9jzx2900000gq/T/ipykernel_66368/3416284705.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inser

In [71]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [72]:
print(len(y))
print(len(X_train), " ", len(X_test))

11042
7398   3644


In [73]:
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [74]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 40.61%


In [75]:
#y_test_df = pd.DataFrame({"Contribution_n1": y_test})
y_test["Predictions"]=y_pred
y_test["Better"] = np.where(y_test['Predictions']<=y_test['Contribution_n1'], 1, 0)
#y_test["Spot On"] = np.where(y_test['Predictions']==y_test['Contribution_n1'], 1, 0)
y_test["Worse"] = np.where(y_test['Predictions']>y_test['Contribution_n1'], 1, 0)
print(sum(list(y_test['Better'])))
print(len(list(y_test['Better'])))
print(round(sum(list(y_test['Better']))/len(list(y_test['Better'])), 2))
#y_test["Difference"] = y_test['Predictions']-y_test['Contribution_n1']
cond = y_test.groupby("Predictions").sum()
cond2 = y_test.groupby(["Contribution_n1", "Predictions"]).count()
cond2


2500
3644
0.69


Better  Worse
Contribution_n1 Predictions               
0               0                36     36
                1                58     58
                2                45     45
                3                24     24
                4                 3      3
1               0                30     30
                1               148    148
                2               158    158
                3                82     82
                4                11     11
                6                 3      3
2               0                15     15
                1                98     98
                2               355    355
                3               316    316
                4                67     67
                5                 5      5
                6                 7      7
3               0                 2      2
                1                36     36
                2               228    228
                3               565    565
                4               265    265
                5                19     19
                6                11     11
4               0                 1      1
                1                12     12
                2                62     62
                3               336    336
                4               292    292
                5                46     46
                6                23     23
5               1                 1      1
                2                 5      5
                3                53     53
                4                92     92
                5                58     58
                6                 1      1
6               1                 3      3
                2                 1      1
                3                11     11
                4                34     34
                6                26     26

In [76]:
cond

,Contribution_n1,Better,Worse
Predictions,,,
0,70,84,0
1,523,298,58
2,1831,651,203
3,4084,965,422
4,2772,418,346
5,541,58,70
6,303,26,45


In [77]:
#frames = [y_test, pred_df]
#comp = pd.concat([y_test, pred_df])
#comp

Transfer Models

In [78]:
df_transfer = df[df['player_type']==0]
#df_transfer['Year_dif'] = df_transfer['year_n1']-df_transfer['year']
#df_transfer
df_test_transfer = df_transfer[['player_type', 'X.Poss', 'X.Poss_n1', 'X.Min',
                'X.Min_n1', 'ORtg', 'Contribution_n1', 'Year_dif', 'stl_per_poss', 
                'blk_per_poss', 'DR.', 'AdjEM_x']]
Xt = df_test_transfer[['X.Poss', 'X.Min', 'X.Min_n1', 'ORtg', 
                      'Year_dif', 'stl_per_poss', 'blk_per_poss', 
                      'DR.', 'AdjEM_x']]
yt = df_test_transfer[['Contribution_n1']]

In [79]:
seed = 7
test_size = 0.33
Xt_train, Xt_test, yt_train, yt_test = train_test_split(Xt, yt, test_size=test_size, random_state=seed)

In [80]:
model = XGBClassifier()
model.fit(Xt_train, yt_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [81]:
yt_pred = model.predict(Xt_test)
predictionst = [round(value) for value in yt_pred]
accuracyt = accuracy_score(yt_test, predictionst)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 40.61%


In [82]:
df.Yr_n1.unique()

array(['Sr', 'Jr', 'So', 'Fr'], dtype=object)

In [83]:
new = pd.DataFrame(zip(df.Ht), columns=['Ht'])
ht_df = new.groupby(['Ht']).size()
ht_df

Ht
62       2
66       5
67      13
68      40
69      59
70     124
71     172
72     471
73     594
74     822
75    1024
76    1107
77    1200
78    1093
79    1215
80    1103
81     921
82     581
83     262
84     139
85      68
86      11
87      10
88       5
89       1
dtype: int64

In [84]:
new1 = pd.DataFrame(zip(df.Wt), columns=['Wt'])
wt_df = new1.groupby(['Wt']).size()
wt_df

Wt
0.0      108
139.0      1
140.0     10
145.0     13
148.0      1
        ... 
295.0      5
299.0      1
300.0      6
305.0      2
350.0      1
Length: 135, dtype: int64

In [91]:
# All In One Go

# Prep initial df
d = pd.read_csv("/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/final_data_by_player_type.csv")
d = pd.merge(d, kp, left_on=['Team', 'year'], right_on=['TeamName', 'Season'], how='left')
d = pd.merge(d, kp, left_on=['Team_n1', 'year'], right_on=['TeamName', 'Season'], how='left')
df = pd.merge(d, kp, left_on=['Team_n1', 'year_n1'], right_on=['TeamName', 'Season'], how='left')
df = pd.merge(df, kp_conf_rat, left_on=['conf_abbr', 'year'], right_on=['Conf_Abb', 'Year_Conf'], how='left')
df = pd.merge(df, kp_conf_rat, left_on=['conf_abbr_n1', 'year'], right_on=['Conf_Abb', 'Year_Conf'], how='left')
df = pd.merge(df, kp_conf_rat, left_on=['conf_abbr_n1', 'year_n1'], right_on=['Conf_Abb', 'Year_Conf'], how='left')
df = df.replace(to_replace="NO IDEA", value=0)
df = df.replace(to_replace="---", value=0)
df = df.fillna(0)

# Convert to Numeric
df[['games', 'games_started', 'mp_per_g', 'fg_per_g', 'fga_per_g', 'fg_pct', 'fg2_per_g', 'fg2a_per_g', 'fg2_pct', 'fg3_per_g', 'fg3a_per_g', 'fg3_pct', 'ft_per_g', 'fta_per_g', 'ft_pct', 'orb_per_g', 'drb_per_g', 'trb_per_g', 'ast_per_g', 'stl_per_g', 'blk_per_g', 'tov_per_g', 'pf_per_g', 'pts_per_g', 'sos', 'mp', 'fg', 'fga', 'fg2', 'fg2a', 'fg2_pct.1', 'fg3', 'fg3a', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'fg_per_min', 'fga_per_min', 'fg2_per_min', 'fg2a_per_min', 'fg3_per_min', 'fg3a_per_min', 'ft_per_min', 'fta_per_min', 'trb_per_min', 'ast_per_min', 'stl_per_min', 'blk_per_min', 'tov_per_min', 'pf_per_min', 'pts_per_min', 'fg_per_poss', 'fga_per_poss', 'fg2_per_poss', 'fg2a_per_poss', 'fg3_per_poss', 'fg3a_per_poss', 'ft_per_poss', 'fta_per_poss', 'trb_per_poss', 'ast_per_poss', 'stl_per_poss', 'blk_per_poss', 'tov_per_poss', 'pf_per_poss', 'pts_per_poss', 'off_rtg', 'def_rtg', 'per', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'pprod', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'ows', 'dws', 'ws', 'ws_per_40', 'obpm', 'dbpm', 'bpm', 'year', 'Ht', 'Wt', 'G', 'S', 'X.Min', 'ORtg', 'X.Poss', 'X.Shots', 'games_n1', 'games_started_n1', 'mp_per_g_n1', 'fg_per_g_n1', 'fga_per_g_n1', 'fg_pct_n1', 'fg2_per_g_n1', 'fg2a_per_g_n1', 'fg2_pct_n1', 'fg3_per_g_n1', 'fg3a_per_g_n1', 'fg3_pct_n1', 'ft_per_g_n1', 'fta_per_g_n1', 'ft_pct_n1', 'orb_per_g_n1', 'drb_per_g_n1', 'trb_per_g_n1', 'ast_per_g_n1', 'stl_per_g_n1', 'blk_per_g_n1', 'tov_per_g_n1', 'pf_per_g_n1', 'pts_per_g_n1', 'sos_n1', 'mp_n1', 'fg_n1', 'fga_n1', 'fg2_n1', 'fg2a_n1', 'fg2_pct.1_n1', 'fg3_n1', 'fg3a_n1', 'ft_n1', 'fta_n1', 'orb_n1', 'drb_n1', 'trb_n1', 'ast_n1', 'stl_n1', 'blk_n1', 'tov_n1', 'pf_n1', 'pts_n1', 'fg_per_min_n1', 'fga_per_min_n1', 'fg2_per_min_n1', 'fg2a_per_min_n1', 'fg3_per_min_n1', 'fg3a_per_min_n1', 'ft_per_min_n1', 'fta_per_min_n1', 'trb_per_min_n1', 'ast_per_min_n1', 'stl_per_min_n1', 'blk_per_min_n1', 'tov_per_min_n1', 'pf_per_min_n1', 'pts_per_min_n1', 'fg_per_poss_n1', 'fga_per_poss_n1', 'fg2_per_poss_n1', 'fg2a_per_poss_n1', 'fg3_per_poss_n1', 'fg3a_per_poss_n1', 'ft_per_poss_n1', 'fta_per_poss_n1', 'trb_per_poss_n1', 'ast_per_poss_n1', 'stl_per_poss_n1', 'blk_per_poss_n1', 'tov_per_poss_n1', 'pf_per_poss_n1', 'pts_per_poss_n1', 'off_rtg_n1', 'def_rtg_n1', 'per_n1', 'ts_pct_n1', 'efg_pct_n1', 'fg3a_per_fga_pct_n1', 'fta_per_fga_pct_n1', 'pprod_n1', 'orb_pct_n1', 'drb_pct_n1', 'trb_pct_n1', 'ast_pct_n1', 'stl_pct_n1', 'blk_pct_n1', 'tov_pct_n1', 'usg_pct_n1', 'ows_n1', 'dws_n1', 'ws_n1', 'ws_per_40_n1', 'obpm_n1', 'dbpm_n1', 'bpm_n1', 'year_n1', 'Ht_n1', 'Wt_n1', 'G_n1', 'S_n1', 'X.Min_n1', 'ORtg_n1', 'X.Poss_n1', 'X.Shots_n1']] = df[['games', 'games_started', 'mp_per_g', 'fg_per_g', 'fga_per_g', 'fg_pct', 'fg2_per_g', 'fg2a_per_g', 'fg2_pct', 'fg3_per_g', 'fg3a_per_g', 'fg3_pct', 'ft_per_g', 'fta_per_g', 'ft_pct', 'orb_per_g', 'drb_per_g', 'trb_per_g', 'ast_per_g', 'stl_per_g', 'blk_per_g', 'tov_per_g', 'pf_per_g', 'pts_per_g', 'sos', 'mp', 'fg', 'fga', 'fg2', 'fg2a', 'fg2_pct.1', 'fg3', 'fg3a', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'fg_per_min', 'fga_per_min', 'fg2_per_min', 'fg2a_per_min', 'fg3_per_min', 'fg3a_per_min', 'ft_per_min', 'fta_per_min', 'trb_per_min', 'ast_per_min', 'stl_per_min', 'blk_per_min', 'tov_per_min', 'pf_per_min', 'pts_per_min', 'fg_per_poss', 'fga_per_poss', 'fg2_per_poss', 'fg2a_per_poss', 'fg3_per_poss', 'fg3a_per_poss', 'ft_per_poss', 'fta_per_poss', 'trb_per_poss', 'ast_per_poss', 'stl_per_poss', 'blk_per_poss', 'tov_per_poss', 'pf_per_poss', 'pts_per_poss', 'off_rtg', 'def_rtg', 'per', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'pprod', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'ows', 'dws', 'ws', 'ws_per_40', 'obpm', 'dbpm', 'bpm', 'year', 'Ht', 'Wt', 'G', 'S', 'X.Min', 'ORtg', 'X.Poss', 'X.Shots', 'games_n1', 'games_started_n1', 'mp_per_g_n1', 'fg_per_g_n1', 'fga_per_g_n1', 'fg_pct_n1', 'fg2_per_g_n1', 'fg2a_per_g_n1', 'fg2_pct_n1', 'fg3_per_g_n1', 'fg3a_per_g_n1', 'fg3_pct_n1', 'ft_per_g_n1', 'fta_per_g_n1', 'ft_pct_n1', 'orb_per_g_n1', 'drb_per_g_n1', 'trb_per_g_n1', 'ast_per_g_n1', 'stl_per_g_n1', 'blk_per_g_n1', 'tov_per_g_n1', 'pf_per_g_n1', 'pts_per_g_n1', 'sos_n1', 'mp_n1', 'fg_n1', 'fga_n1', 'fg2_n1', 'fg2a_n1', 'fg2_pct.1_n1', 'fg3_n1', 'fg3a_n1', 'ft_n1', 'fta_n1', 'orb_n1', 'drb_n1', 'trb_n1', 'ast_n1', 'stl_n1', 'blk_n1', 'tov_n1', 'pf_n1', 'pts_n1', 'fg_per_min_n1', 'fga_per_min_n1', 'fg2_per_min_n1', 'fg2a_per_min_n1', 'fg3_per_min_n1', 'fg3a_per_min_n1', 'ft_per_min_n1', 'fta_per_min_n1', 'trb_per_min_n1', 'ast_per_min_n1', 'stl_per_min_n1', 'blk_per_min_n1', 'tov_per_min_n1', 'pf_per_min_n1', 'pts_per_min_n1', 'fg_per_poss_n1', 'fga_per_poss_n1', 'fg2_per_poss_n1', 'fg2a_per_poss_n1', 'fg3_per_poss_n1', 'fg3a_per_poss_n1', 'ft_per_poss_n1', 'fta_per_poss_n1', 'trb_per_poss_n1', 'ast_per_poss_n1', 'stl_per_poss_n1', 'blk_per_poss_n1', 'tov_per_poss_n1', 'pf_per_poss_n1', 'pts_per_poss_n1', 'off_rtg_n1', 'def_rtg_n1', 'per_n1', 'ts_pct_n1', 'efg_pct_n1', 'fg3a_per_fga_pct_n1', 'fta_per_fga_pct_n1', 'pprod_n1', 'orb_pct_n1', 'drb_pct_n1', 'trb_pct_n1', 'ast_pct_n1', 'stl_pct_n1', 'blk_pct_n1', 'tov_pct_n1', 'usg_pct_n1', 'ows_n1', 'dws_n1', 'ws_n1', 'ws_per_40_n1', 'obpm_n1', 'dbpm_n1', 'bpm_n1', 'year_n1', 'Ht_n1', 'Wt_n1', 'G_n1', 'S_n1', 'X.Min_n1', 'ORtg_n1', 'X.Poss_n1', 'X.Shots_n1']].apply(pd.to_numeric, errors="ignore")

# Add Contributions
conditions = [(df['X.Poss_n1']>=28.0), (df['X.Poss_n1']>=24.0), (df['X.Poss_n1']>=20.0), (df['X.Poss_n1']>=16), 
              (df['X.Poss_n1']>=12.0), (df['X.Poss_n1']<12)]
               # & (df['X.Min_n1']>=10.0), (df['X.Min_n1']<10)]
#values = ['Go-To Guy', 'Major Contributor', 'Significant Contributor', 'Role Players', 'Limited Role', 'Nearly Invisible', 'Bench Warmer']
values = [0, 1, 2, 3, 4, 5]
df['Contribution_n1'] = np.select(conditions, values)

# Add Year in School KP
year_cond = [(df['Yr_n1']=='Fr'), (df['Yr_n1']=='So'), 
             (df['Yr_n1']=='Jr'), (df['Yr_n1']=='Sr')]
year_val = [1, 2, 3, 4]
df['Class'] = np.select(year_cond, year_val)

# Add Ht. Categories
ht_cond = [(df['Ht']<=74), (df['Ht']>74) & (df['Ht']<=76), (df['Ht']>76) & (df['Ht']<=78),
           (df['Ht']>78) & (df['Ht']<=80), (df['Ht']>80)]

ht_val = [0, 1, 2, 3, 4]
df['Pos_Ht'] = np.select(ht_cond, ht_val)
#df[‘column’] =( df[‘column’] – df[‘column’].mean() ) / df[‘column’].std()


# Additional Features + Model Prep
df = df.drop(df[(df['games']<=14) & (df['games_started']/df['games']>0.25)].index)
df['Year_dif'] = df['year_n1']-df['year']
df['Team_AdjEM_dif'] = df['AdjEM_y']-df['AdjEM_x']
df['KP_Rk_Dif'] = df['RankAdjEM_y']-df['RankAdjEM_x']
df['KP_Conf_Dif'] = df['Conf_Rating_y']-df['Conf_Rating_x']
df['Calc A'] = df['X.Min']*df['ORtg']
df['Calc B'] = df['stl_per_poss']+df['blk_per_poss']+df['DR.']
# Trying to flag red 
df_test1 = df[['games', 'games_started', 'player_type', 'KP_Conf_Dif', 'X.Poss',  'X.Min',
                 'ORtg', 'Contribution_n1', 'Year_dif', 'stl_per_poss', 
                'blk_per_poss', 'DR.', 'KP_Rk_Dif', 'Calc A', 'Team_AdjEM_dif', 
                'Calc B', 'Class', 'Pos_Ht', 'Ht', 'Wt']]

#'Team_AdjEM_dif', 'X.Min_n1', 'X.Poss_n1',
#df_test1 = df_test1[df_test1['player_type']==0]
columns = list(df_test1.columns)
columns.remove('Contribution_n1')
#columns.remove('player_type')
keep_col = []
rem_columns = columns
check = True
accuracy_end = 0
while len(rem_columns)>0 and check == True:
    acc_check = 0

    for col in rem_columns:
        X = df_test1[keep_col + [col]]
    # X = df_test1[['player_type', 'X.Poss', 'X.Min', 'X.Min_n1', 'ORtg', 
    #                       'Year_dif', 'stl_per_poss', 'blk_per_poss', 
    #                       'DR.', 'KP_Rk_Dif', 'Calc A']]
        y = df_test1[['Contribution_n1']]

        # Model
        seed = 7
        test_size = 0.2
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
        model = XGBClassifier()
        model.fit(X_train, y_train)

        # Find Accuracy
        y_pred = model.predict(X_test)
        predictions = [round(value) for value in y_pred]
        accuracy = accuracy_score(y_test, predictions)
        print(keep_col + [col])
        print("Accuracy: ", accuracy * 100)
        if accuracy > acc_check:
            acc_check = accuracy
            col_to_add = col
            check = True
    #print("Adding: ", col_to_add)
    
    print("Rem Columns: ", rem_columns)
    #keep_col = keep_col + [col_to_add]
    if accuracy_end < acc_check:
        print("Adding: ", col_to_add)
        print("Rem Columns: ", rem_columns)
        accuracy_end = acc_check
        rem_columns.remove(col_to_add)
        keep_col = keep_col + [col_to_add]
    else:
        print("Fails", accuracy_end, acc_check)
        check = False
    print(keep_col)
    

/var/folders/cq/16hkxgp926d466c0g9jzx2900000gq/T/ipykernel_66368/1880978974.py:4: DtypeWarning: Columns (13,16,38,144,147,169) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv("/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/final_data_by_player_type.csv")
/var/folders/cq/16hkxgp926d466c0g9jzx2900000gq/T/ipykernel_66368/1880978974.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Contribution_n1'] = np.select(conditions, values)
/var/folders/cq/16hkxgp926d466c0g9jzx2900000gq/T/ipykernel_66368/1880978974.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

['games']
Accuracy:  31.159080244016895
['games_started']
Accuracy:  30.17362740497419
['player_type']
Accuracy:  31.628343500703892
['KP_Conf_Dif']
Accuracy:  30.64289066166119
['X.Poss']
Accuracy:  42.28061942749883
['X.Min']
Accuracy:  29.094321914594083
['ORtg']
Accuracy:  27.451900516189582
['Year_dif']
Accuracy:  31.58141717503519
['stl_per_poss']
Accuracy:  30.689816987329895
['blk_per_poss']
Accuracy:  31.065227592679495
['DR.']
Accuracy:  30.783669638667295
['KP_Rk_Dif']
Accuracy:  31.018301267010795
['Calc A']
Accuracy:  29.798216799624587
['Team_AdjEM_dif']
Accuracy:  31.206006569685595
['Calc B']
Accuracy:  28.343500703894886
['Class']
Accuracy:  31.628343500703892
['Pos_Ht']
Accuracy:  31.628343500703892
['Ht']
Accuracy:  31.58141717503519
['Wt']
Accuracy:  31.44063819802909
Rem Columns:  ['games', 'games_started', 'player_type', 'KP_Conf_Dif', 'X.Poss', 'X.Min', 'ORtg', 'Year_dif', 'stl_per_poss', 'blk_per_poss', 'DR.', 'KP_Rk_Dif', 'Calc A', 'Team_AdjEM_dif', 'Calc B', '

In [117]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import neighbors

df_test1 = df[['games', 'games_started',  'KP_Conf_Dif', 'X.Poss',  'X.Min',
                 'ORtg', 'Contribution_n1', 'Year_dif', 'stl_per_poss', 
                'blk_per_poss', 'DR.', 'Calc A', 'Team_AdjEM_dif', 
                'Calc B', 'Class', 'Pos_Ht', 'Ht', 'Wt']]
#'player_type', 'KP_Rk_Dif',
#'Team_AdjEM_dif', 'X.Min_n1', 'X.Poss_n1',
#df_test1 = df_test1[df_test1['player_type']==0]
columns = list(df_test1.columns)
columns.remove('Contribution_n1')
#columns.remove('player_type')
keep_col = []
rem_columns = columns
check = True
accuracy_end = 0
while len(rem_columns)>0 and check == True:
    acc_check = 0

    for col in rem_columns:
        X = df_test1[keep_col + [col]]
    # X = df_test1[['player_type', 'X.Poss', 'X.Min', 'X.Min_n1', 'ORtg', 
    #                       'Year_dif', 'stl_per_poss', 'blk_per_poss', 
    #                       'DR.', 'KP_Rk_Dif', 'Calc A']]
        y = df_test1[['Contribution_n1']]

        # Model
        seed = 7
        test_size = 0.1
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
        # Random Forrest
        #model = RandomForestClassifier()
        #model.fit(X_train, y_train)

        # Naive-Bayes
        # model = GaussianNB()
        # model.fit(X_train, y_train)

        # Nearest Neighbors
        model = neighbors.KNeighborsClassifier()
        model.fit(X_train, y_train)

        # Find Accuracy
        accuracy = model.score(X_test, y_test)
        # y_pred = model.predict(X_test)
        # predictions = [round(value) for value in y_pred]
        # accuracy = accuracy_score(y_test, predictions)
        print(keep_col + [col])
        print("Accuracy: ", accuracy * 100)
        if accuracy > acc_check:
            acc_check = accuracy
            col_to_add = col
            check = True
    #print("Adding: ", col_to_add)
    
    print("Rem Columns: ", rem_columns)
    #keep_col = keep_col + [col_to_add]
    if accuracy_end < acc_check:
        print("Adding: ", col_to_add)
        print("Rem Columns: ", rem_columns)
        accuracy_end = acc_check
        rem_columns.remove(col_to_add)
        keep_col = keep_col + [col_to_add]
    else:
        print("Fails", accuracy_end, acc_check)
        check = False
    print(keep_col)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n

['games']
Accuracy:  26.07879924953096
['games_started']
Accuracy:  25.797373358348967
['KP_Conf_Dif']
Accuracy:  23.733583489681052
['X.Poss']
Accuracy:  34.146341463414636
['X.Min']
Accuracy:  26.360225140712945
['ORtg']
Accuracy:  25.70356472795497
['Year_dif']
Accuracy:  21.575984990619137
['stl_per_poss']
Accuracy:  25.234521575984992
['blk_per_poss']
Accuracy:  25.70356472795497
['DR.']
Accuracy:  26.547842401500937


/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n

['Calc A']
Accuracy:  23.827392120075046
['Team_AdjEM_dif']
Accuracy:  24.296435272045027
['Calc B']
Accuracy:  23.076923076923077
['Class']
Accuracy:  19.9812382739212
['Pos_Ht']
Accuracy:  13.696060037523452
['Ht']
Accuracy:  20.356472795497186
['Wt']
Accuracy:  22.88930581613508
Rem Columns:  ['games', 'games_started', 'KP_Conf_Dif', 'X.Poss', 'X.Min', 'ORtg', 'Year_dif', 'stl_per_poss', 'blk_per_poss', 'DR.', 'Calc A', 'Team_AdjEM_dif', 'Calc B', 'Class', 'Pos_Ht', 'Ht', 'Wt']
Adding:  X.Poss
Rem Columns:  ['games', 'games_started', 'KP_Conf_Dif', 'X.Poss', 'X.Min', 'ORtg', 'Year_dif', 'stl_per_poss', 'blk_per_poss', 'DR.', 'Calc A', 'Team_AdjEM_dif', 'Calc B', 'Class', 'Pos_Ht', 'Ht', 'Wt']
['X.Poss']
['X.Poss', 'games']
Accuracy:  34.709193245778614
['X.Poss', 'games_started']
Accuracy:  35.74108818011257
['X.Poss', 'KP_Conf_Dif']
Accuracy:  33.30206378986867
['X.Poss', 'X.Min']
Accuracy:  33.58348968105066


/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n

...

/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n

['X.Poss', 'DR.', 'games']
Accuracy:  34.80300187617261
['X.Poss', 'DR.', 'games_started']
Accuracy:  35.92870544090056
['X.Poss', 'DR.', 'KP_Conf_Dif']
Accuracy:  38.08630393996248
['X.Poss', 'DR.', 'X.Min']
Accuracy:  34.61538461538461
['X.Poss', 'DR.', 'ORtg']
Accuracy:  32.926829268292686
['X.Poss', 'DR.', 'Year_dif']
Accuracy:  37.89868667917448
['X.Poss', 'DR.', 'stl_per_poss']
Accuracy:  35.74108818011257
['X.Poss', 'DR.', 'blk_per_poss']
Accuracy:  35.17823639774859
['X.Poss', 'DR.', 'Calc A']
Accuracy:  29.45590994371482
['X.Poss', 'DR.', 'Team_AdjEM_dif']
Accuracy:  40.150093808630395
['X.Poss', 'DR.', 'Calc B']
Accuracy:  34.427767354596625


/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n

['X.Poss', 'DR.', 'Class']
Accuracy:  33.771106941838646
['X.Poss', 'DR.', 'Pos_Ht']
Accuracy:  34.427767354596625
['X.Poss', 'DR.', 'Ht']
Accuracy:  34.052532833020635
['X.Poss', 'DR.', 'Wt']
Accuracy:  33.95872420262664
Rem Columns:  ['games', 'games_started', 'KP_Conf_Dif', 'X.Min', 'ORtg', 'Year_dif', 'stl_per_poss', 'blk_per_poss', 'Calc A', 'Team_AdjEM_dif', 'Calc B', 'Class', 'Pos_Ht', 'Ht', 'Wt']
Adding:  Team_AdjEM_dif
Rem Columns:  ['games', 'games_started', 'KP_Conf_Dif', 'X.Min', 'ORtg', 'Year_dif', 'stl_per_poss', 'blk_per_poss', 'Calc A', 'Team_AdjEM_dif', 'Calc B', 'Class', 'Pos_Ht', 'Ht', 'Wt']
['X.Poss', 'DR.', 'Team_AdjEM_dif']
['X.Poss', 'DR.', 'Team_AdjEM_dif', 'games']
Accuracy:  34.8968105065666
['X.Poss', 'DR.', 'Team_AdjEM_dif', 'games_started']
Accuracy:  35.74108818011257
['X.Poss', 'DR.', 'Team_AdjEM_dif', 'KP_Conf_Dif']
Accuracy:  37.61726078799249
['X.Poss', 'DR.', 'Team_AdjEM_dif', 'X.Min']
Accuracy:  36.67917448405254
['X.Poss', 'DR.', 'Team_AdjEM_dif', '

/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n